In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

docs_path = Path.cwd().joinpath("docs")

md_files = list(docs_path.glob("**/*.md"))

[PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/install.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/index.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/community/welcome.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/community/contribute.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/explanations/what-is-rag.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/examples/README.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/python-api.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/deploy.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/config.md'),
 PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/release-notes.md'),
 Po

In [9]:
!pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [11]:
# openai work
from openai import OpenAI

client = OpenAI()
def stream_openai(client, role, message):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": role, "content": message}],
        stream=True,
    )
    message = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            message += chunk.choices[0].delta.content
    return message

m = stream_openai(client, "user", "hello, how are you?")
m

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [23]:
from typing import Optional
from ragna.source_storages import RagnaDemoSourceStorage
from ragna.assistants import RagnaDemoAssistant
from ragna.preprocessors import RagnaDemoPreprocessor
from ragna import Rag
from ragna.core import (
    LocalDocument, Document, QueryPreprocessor, MetadataFilter, ProcessedQuery
)
from ragna.assistants import Gpt4



storage = RagnaDemoSourceStorage()

documents = [
            (
                document
                if isinstance(document, Document)
                else LocalDocument.from_path(document)
            )
            for document in md_files
        ]
storage.store("ragna_docs", documents)


class TestPreprocessor(QueryPreprocessor):

    def __init__(self):
        # self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = ("take the following prompt and "
                       "improve it so that it will be "
                       "better for querying a database: " + prompt)
        
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        processed_query = self.ask_assistant(query)
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )


chat = Rag().chat(
    input=None,
    source_storage = storage,
    assistant=RagnaDemoAssistant,
    preprocessor=TestPreprocessor,
    corpus_name="ragna_docs"
)

In [24]:
_ = await chat.prepare()

In [25]:
print(await chat.answer("What happened last year?"))

I'm a demo assistant and can be used to try Ragna's workflow.
I will only mirror back my inputs. 

So far I have received 1 messages.

Your last prompt was:

> To improve the prompt for querying a database, it's important to specify the context, the type of events you're interested in, and any relevant parameters. Here’s a refined version:

"Can you provide a summary of significant events that occurred in the year 2022, including their dates, locations, and categories (e.g., economic, social, political, or environmental)?"

This version clarifies the timeframe and the specific details you want to obtain, which will facilitate more accurate database queries.

These are the sources I was given:

- install.md: # Installation ## Prerequisites You need Python 3.9 or above in your working environment to [...]
- index.md: --- title: Home --- # Ragna — Open source RAG orchestration framework With an intuitive API [...]
- welcome.md: # Welcome! Thanks for participating in the Ragna community! A